### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head(1)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53


## Player Count

* Display the total number of players


In [2]:
total_players = len(purchase_data["SN"].unique())
total_list = [total_players]
total_columns = ["Total Players"]
total_players = pd.DataFrame([total_list],columns=total_columns)
total_players

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
unique_items = len(purchase_data["Item ID"].unique())
average_price = purchase_data["Price"].mean()
num_purchases = purchase_data["Purchase ID"].count()
total_rev = purchase_data["Price"].sum()

summ_list = [unique_items, average_price, num_purchases, total_rev]
summ_col = ["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"]

summ_df = pd.DataFrame([summ_list], columns=summ_col)

summ_df["Average Price"] = summ_df["Average Price"].map("${:,.2f}".format)
summ_df["Total Revenue"] = summ_df["Total Revenue"].map("${:,.2f}".format)

summ_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
purch_users_df = purchase_data[["Gender", "SN"]].drop_duplicates()
purch_users_df.count()

purch_group = purch_users_df.groupby(["Gender"])
#purch_group.describe()

summ_demo_df = purch_group.count()
summ_demo_df = summ_demo_df.rename(columns={"SN": "Total Count"})
summ_demo_df
summ_demo_df["Percentage of Players"] = summ_demo_df["Total Count"] / purch_users_df["SN"].count() * 100
summ_demo_df["Percentage of Players"] = summ_demo_df["Percentage of Players"].map("{:.2f}%".format)
summ_demo_df

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
purch_group_2 = purchase_data.groupby("Gender")
gender_count_df = pd.DataFrame(purch_group_2.count()["SN"])
gender_count_df = gender_count_df.rename(columns={"SN": "Purchase Count"})
gender_count_df
gender_price_avg_df = pd.DataFrame(purch_group_2.mean()["Price"])
gender_price_avg_df = gender_price_avg_df.rename(columns={"Price": "Average Purchase Price"})
gender_price_avg_df
total_purchase_df = pd.DataFrame(purch_group_2.sum()["Price"])
total_purchase_df = total_purchase_df.rename(columns={"Price": "Total Purchase Value"})
total_purchase_df
summ_analysis_df = pd.merge(gender_count_df, gender_price_avg_df, on="Gender")
summ_analysis_df
summ_analysis_df = pd.merge(summ_analysis_df, total_purchase_df, on="Gender")
summ_analysis_df

total_purchase_avg_df = pd.DataFrame(total_purchase_df["Total Purchase Value"]
                                     / summ_demo_df["Total Count"], columns=["Avg Total Purchase Per Person"])
total_purchase_avg_df

summ_analysis_df = pd.merge(summ_analysis_df, total_purchase_avg_df, on="Gender")
summ_analysis_df["Average Purchase Price"] = summ_analysis_df["Average Purchase Price"].map("${:,.2f}".format)
summ_analysis_df["Total Purchase Value"] = summ_analysis_df["Total Purchase Value"].map("${:,.2f}".format)
summ_analysis_df["Avg Total Purchase Per Person"] = summ_analysis_df["Avg Total Purchase Per Person"].map("${:,.2f}".format)

summ_analysis_df



,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [30]:
age_bins_1 = [0]
age_bins_2 = np.arange(9, 44, 5).tolist()
age_bins_3 = [1024]
age_bins = age_bins_1 + age_bins_2 + age_bins_3
age_bins

age_bin_labels = []

for age_loop in range(len(age_bins) - 1):
    if age_loop == 0:
        age_bin_labels.append(f"<{age_bins[age_loop + 1] + 1}")
    elif age_loop < (len(age_bins) - 2):
        age_bin_labels.append(f"{age_bins[age_loop] + 1}-{age_bins[age_loop + 1]}")
    else:
        age_bin_labels.append(f"{age_bins[age_loop] + 1}+")
    
age_bin_labels

purchase_unique = purchase_data.drop_duplicates(subset=["SN"]).copy()
#purchase_unique.count()

purchase_unique["Age Bracket"] = pd.cut(purchase_unique["Age"], age_bins, labels=age_bin_labels)
#print(purchase_unique["Age Bracket"].value_counts())
#age_bins
bin_group = purchase_unique.groupby(["Age Bracket"])
bin_age_count = pd.DataFrame(bin_group.count()["SN"])
bin_age_count = bin_age_count.rename(columns={"SN": "Total Count"})
bin_age_count["Percentage of Players"] = bin_age_count["Total Count"] / purchase_unique["SN"].count() * 100
bin_age_count["Percentage of Players"] = bin_age_count["Percentage of Players"].map("{:,.2f}%".format)
bin_age_count
    

,Total Count,Percentage of Players
Age Bracket,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [45]:
purchase_nonunique = purchase_data.copy()

purchase_nonunique["Age Bracket"] = pd.cut(purchase_nonunique["Age"], age_bins, labels=age_bin_labels)

bin_group_nu = purchase_nonunique.groupby(["Age Bracket"])
bin_age_count_nu = pd.DataFrame(bin_group_nu.count()["SN"])
bin_age_count_nu = bin_age_count_nu.rename(columns={"SN": "Purchase Count"})

bin_age_avg_purch_nu = pd.DataFrame(bin_group_nu.mean()["Price"])
bin_age_avg_purch_nu = bin_age_avg_purch_nu.rename(columns={"Price": "Average Purchase Price"})
bin_age_avg_purch_nu

bin_age_total_purch_nu = pd.DataFrame(bin_group_nu.sum()["Price"])
bin_age_total_purch_nu = bin_age_total_purch_nu.rename(columns={"Price": "Total Purchase Value"})
bin_age_total_purch_nu

summ_age_df = pd.merge(bin_age_count_nu, bin_age_avg_purch_nu, on="Age Bracket")
summ_age_df

summ_age_df = pd.merge(summ_age_df, bin_age_total_purch_nu, on="Age Bracket")
summ_age_df

summ_age_df["Avg Total Purchase per Person"] = summ_age_df["Total Purchase Value"] / bin_age_count["Total Count"]
summ_age_df["Average Purchase Price"] = summ_age_df["Average Purchase Price"].map("${:,.2f}".format)
summ_age_df["Total Purchase Value"] = summ_age_df["Total Purchase Value"].map("${:,.2f}".format)
summ_age_df["Avg Total Purchase per Person"] = summ_age_df["Avg Total Purchase per Person"].map("${:,.2f}".format)
summ_age_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Bracket,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [75]:
purch_group_users = purchase_data.groupby("SN")

user_purch_count_df = pd.DataFrame(purch_group_users.count()["Purchase ID"])
#user_purch_count_df.sort_values("Purchase ID", ascending=False).head()
user_purch_count_df = user_purch_count_df.rename(columns={"Purchase ID": "Purchase Count"})
user_purch_count_df.sort_values("Purchase Count", ascending=False).head()

user_price_avg_df = pd.DataFrame(purch_group_users.mean()["Price"])
user_price_avg_df = user_price_avg_df.rename(columns={"Price": "Average Purchase Price"})

user_price_total_df = pd.DataFrame(purch_group_users.sum()["Price"])
user_price_total_df = user_price_total_df.rename(columns={"Price": "Total Purchase Value"})

user_price_total_df.head()

summ_user_df = pd.merge(user_purch_count_df, user_price_avg_df, on="SN")
summ_user_df = pd.merge(summ_user_df, user_price_total_df, on="SN")

summ_user_df["Average Purchase Price"] = summ_user_df["Average Purchase Price"].map("${:,.2f}".format)
summ_user_df["Total Purchase Value"] = summ_user_df["Total Purchase Value"].map("${:,.2f}".format)
summ_user_df.sort_values(["Total Purchase Value"], ascending=False).head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Haillyrgue51,3,$3.17,$9.50
Phistym51,2,$4.75,$9.50
Lamil79,2,$4.64,$9.29
Aina42,3,$3.07,$9.22
Saesrideu94,2,$4.59,$9.18


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [121]:
item_df = purchase_data[["Item ID", "Item Name", "Price"]]
item_df.head()

item_group = item_df.groupby(["Item ID", "Item Name"])

item_purch_count_df = pd.DataFrame(item_group.count()["Price"])
item_purch_count_df = item_purch_count_df.rename(columns={"Price": "Purchase Count"})
item_purch_count_df

item_unique_df = item_df.drop_duplicates().set_index(["Item ID", "Item Name"])
item_unique_df

summ_item_df = pd.merge(item_purch_count_df, item_unique_df, on=["Item ID", "Item Name"])
summ_item_df

summ_item_df["Total Purchase Value"] = summ_item_df["Purchase Count"] * summ_item_df["Price"]
summ_item_df.sort_values(["Purchase Count"], ascending=False).head()


,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

